In [1]:
# !pip install -q gradio opencv-python facenet-pytorch albumentations numpy

In [2]:
# !pip install --upgrade numpy pandas

# Setting up UI

In [3]:
import gradio as gr
import time
import cv2
import numpy as np

In [4]:
def add_face():
    cap = cv2.VideoCapture(0)
    
    for _ in range(5):
        ret, temp_frame = cap.read()
        if not ret:
            break
            
    frames = []
    start_time = time.time()
    duration = 6
    interval = 0.5
    while time.time() - start_time < duration:
        ret, frame = cap.read()
        if ret :
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
            time.sleep(interval)
    cap.release()
    return frames

In [5]:
def unlock_face():
    cap = cv2.VideoCapture(0)

    for _ in range(5):
        ret, temp_frame = cap.read()
        if not ret:
            break
            
    ret, frame = cap.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    cap.release()
    return frame

# Face allignment

In [6]:
# !pip install -q facenet-pytorch albumentations

In [7]:
from facenet_pytorch import MTCNN
import torch
from PIL import Image

In [8]:
mtcnn = MTCNN(image_size = 160 , margin = 0 , min_face_size = 40)

In [9]:
def align_face(image):
    pil_image = Image.fromarray(image)
    alligned_face = mtcnn(pil_image)
    if alligned_face is None:
        print("No face detected")
        return None
    return alligned_face

# Data Augmentation

In [10]:
# !pip install --upgrade albumentations

In [11]:
import albumentations as A

In [12]:
def augment_face(image):
    transforms = {
        "rotations" : A.Rotate(limit = 15 , p = 1.0),
        "brightness" : A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1.0),
        "blur" : A.Blur(blur_limit=3, p=1.0),
        "shadow" : A.RandomGamma(gamma_limit=(80, 120), p=1.0)
    }

    augmented_images = []

    for name, transform in transforms.items():
        augmented = transform(image = image)
        augmented_images.append(augmented["image"])

    return augmented_images

# FaceNet embeddings

In [13]:
from facenet_pytorch import InceptionResnetV1

In [14]:
resnet = InceptionResnetV1(pretrained = 'vggface2').eval()

In [15]:
from torchvision import transforms

def get_facenet_embedding(aligned_face):
    
    if aligned_face is None:
        return None

    
    if isinstance(aligned_face, torch.Tensor):
        aligned_face = transforms.ToPILImage()(aligned_face)

    
    preprocess = transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
    ])
    
    face_tensor = preprocess(aligned_face).unsqueeze(0)

    
    with torch.no_grad():
        embedding = resnet(face_tensor)
        embedding = embedding / embedding.norm(p=2)  

    return embedding.cpu().numpy()

# Adding new face

In [16]:
def enroll_pipeline(captured_images):
    enrollment_embeddings = []
    
    for img in captured_images:

        aligned_face = align_face(img)
        if aligned_face is None:
            continue
        
        if isinstance(aligned_face, torch.Tensor):
            aligned_face = transforms.ToPILImage()(aligned_face.cpu())
        

        embedding = get_facenet_embedding(aligned_face)
        if embedding is not None:
            enrollment_embeddings.append(embedding)
        
        
        aligned_np = np.array(aligned_face)
        if aligned_np.dtype != np.uint8:
            
            aligned_np = (aligned_np * 255).astype(np.uint8)
        
        augmented_images = augment_face(aligned_np)
        
        
        for aug_img in augmented_images:
            
            aug_img = np.clip(aug_img, 0, 255).astype(np.uint8)
            try:
                pil_aug_img = Image.fromarray(aug_img)
            except Exception as e:
                print("Error converting augmented image to PIL Image:", e)
                continue
            aug_embedding = get_facenet_embedding(pil_aug_img)
            if aug_embedding is not None:
                enrollment_embeddings.append(aug_embedding)
    
    return enrollment_embeddings

# Unlocking with face (Authentication)

In [17]:
def cosine_similarity(emb1 , emb2):
    return np.dot(emb1, emb2.T) / (np.linalg.norm(emb1)*np.linalg.norm(emb2))

In [18]:
def authenticate(enrollment_embeddings, live_image, threshold = 0.8):
    
    aligned_live = align_face(live_image)
    
    if aligned_live is None:
        print("No face detected in live image.")
        return False
        
    live_embeddings = get_facenet_embedding(aligned_live)
    if live_embeddings is None:
        return False

    for emb in enrollment_embeddings:
        similarity = cosine_similarity(live_embeddings.flatten(), emb.flatten())
        if similarity >= threshold:
            return True
    return False

# Gradio interface

In [19]:
enrollment_embeddings = None

def enrollment_mode():
    global enrollment_embeddings
    
    captured_images = add_face()
    
    enrollment_embeddings = enroll_pipeline(captured_images)
    

def authentication_mode():
   
    live_image = unlock_face()
    
    if enrollment_embeddings is None:
        return "No enrollment data available. Please enroll a face first."
    
    match_found = authenticate(enrollment_embeddings, live_image, threshold=0.8)
    
    if match_found:
        return "Unlocked! Face authenticated." 
    else:
        return "Authentication failed. Face not recognized."

def face_recognition_system(mode):

    if mode == "Enroll Face":
        return enrollment_mode()
    elif mode == "Unlock Face":
        return authentication_mode()
    else:
        return "Invalid mode selected."

In [20]:
custom_css = """
.gradio-container {
  background-color: #000000 !important;
  color: #ffffff !important; /* Default text color: white */
}

.gr-button, button {
  background: linear-gradient(45deg, #FFA500, #FFFF00) !important;
  color: #000000 !important; /* Black text on the gradient */
  border: none !important;
  font-weight: 500 !important;
  cursor: pointer !important;
}

h1, h2, .title, .description, label {
  color: #FFA500 !important;
}
"""

In [21]:
import gradio as gr

In [22]:
def clear_ui():
    return gr.update(value=None), ""

In [23]:
with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("# Face Recognition System")
    gr.Markdown(
        "Select 'Enroll Face' to capture and process images to enroll your face. "
        "Select 'Unlock Face' to authenticate."
    )
    
    mode = gr.Radio(["Enroll Face", "Unlock Face"], label="Select Mode")
    

    with gr.Row():
        submit_btn = gr.Button("Submit")
        clear_btn = gr.Button("Clear")
    
    
    output = gr.Textbox(label="Output")
    
    
    submit_btn.click(fn=face_recognition_system, inputs=mode, outputs=output)
    clear_btn.click(fn=clear_ui, inputs=None, outputs=[mode, output])

In [25]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


[ WARN:0@10.932] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0@25.072] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0@57.564] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0@72.319] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV

#### 